In [1]:
import libsbml
import pickle
import csv
from hashlib import md5
import sys
import os

sys.path.append(os.path.join(os.path.dirname('__file__'), '..'))
import genCache
import rpSBML

In [5]:
cache = genCache.Cache('/home/mdulac/workspace/rpFBA/input_cache/')

In [6]:
cache.all()

In [88]:
#create a new instance of cache to use the depreceatedMNXM_mnxm
deprecatedMNXM_mnxm = pickle.load(open('cache/deprecatedMNXM_mnxm.pickle', 'rb'))

In [89]:
def checklibSBML(value, message):
    if value is None:
        print('LibSBML returned a null value trying to ' + message + '.')
        #raise SystemExit('LibSBML returned a null value trying to ' + message + '.')
        return None
    elif type(value) is int:
        if value==libsbml.LIBSBML_OPERATION_SUCCESS:
            return
        else:
            err_msg = 'Error encountered trying to ' + message + '.' \
                    + 'LibSBML returned error code ' + str(value) + ': "' \
                    + libsbml.OperationReturnValue_toString(value).strip() + '"' 
            print(err_msg)
            raise SystemExit(err_msg)
    else:
        print(message)
        return None

In [90]:
#open full model
yeast_document_ori = libsbml.readSBML('/home/mdulac/Downloads/bigg_iMM904.COBRA-sbml3.xml')
rp_document = libsbml.readSBML('/home/mdulac/Documents/rpFBA/sbml_models/ibisba_test_format.sbml')

In [91]:
##### manually merge two SBML #####
rp_model = rp_document.getModel()
yeast_model_ori = yeast_document_ori.getModel()

In [2]:
path_id = 1
steps = [{'right': {'CMPD_0000000003': 1, 'MNXM13': 1, 'MNXM15': 1, 'MNXM8': 1}, 'left': {'MNXM10': 1, 'MNXM188': 1, 'MNXM4': 1, 'MNXM1': 3}}, {'right': {'TARGET_0000000001': 1, 'MNXM1': 2}, 'left': {'CMPD_0000000003': 1, 'MNXM4': 1, }}]
reaction_smiles = ['[H]Oc1c([H])c([H])c([H])c([H])c1O[H]>>O=O.[H]N=C(O[H])C1=C([H])N(C2([H])OC([H])(C([H])([H])OP(=O)(O[H])OP(=O)(O[H])OC([H])([H])C3([H])OC([H])(n4c([H])nc5c(N([H])[H])nc([H])nc54)C([H])(O[H])C3([H])O[H])C([H])(O[H])C2([H])O[H])C([H])=C([H])C1([H])[H].[H]OC(=O)c1c([H])c([H])c([H])c([H])c1N([H])[H]', '[H]OC(=O)C([H])=C([H])C([H])=C([H])C(=O)O[H]>>O=O.[H]Oc1c([H])c([H])c([H])c([H])c1O[H]', ]
rp_smiles = {'MNXM10': '[H]N=C(O[H])C1=C([H])N(C2([H])OC([H])(C([H])([H])OP(=O)(O[H])OP(=O)(O[H])OC([H])([H])C3([H])OC([H])(n4c([H])nc5c(N([H])[H])nc([H])nc54)C([H])(O[H])C3([H])O[H])C([H])(O[H])C2([H])O[H])C([H])=C([H])C1([H])[H]',
'MNXM188': '[H]OC(=O)c1c([H])c([H])c([H])c([H])c1N([H])[H]',
'MNXM4': 'O=O',
'MNXM1': '[H+]',
'CMPD_0000000003': '[H]Oc1c([H])c([H])c([H])c([H])c1O[H]',
'MNXM13': 'O=C=O',
'MNXM15': '[H]N([H])[H]',
'MNXM8': 'NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)OP([O-])(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O',
'TARGET_0000000001': '[H]OC(=O)C([H])=C([H])C([H])=C([H])C(=O)O[H]'}
rp_inchi = {'MNXM10': 'InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1,3-4,7-8,10-11,13-16,20-21,29-32H,2,5-6H2,(H2,23,33)(H,34,35)(H,36,37)(H2,22,24,25)/p-2/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1',
'MNXM188': 'InChI=1S/C7H7NO2/c8-6-4-2-1-3-5(6)7(9)10/h1-4H,8H2,(H,9,10)/p-1',
'MNXM4': 'InChI=1S/O2/c1-2',
'MNXM1': 'InChI=1S',
'CMPD_0000000003': None,
'MNXM13': 'InChI=1S/CO2/c2-1-3',
'MNXM15': 'InChI=1S/H3N/h1H3/p+1',
'MNXM8': 'InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p-1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1',
'TARGET_0000000001': 'InChI=1S/C6H6O4/c7-5(8)3-1-2-4-6(9)10/h1-4H,(H,7,8)(H,9,10)'}

In [135]:
def compareModels(modelOne, modelTwo):
    for one_species in modelOne.getListOfSpecies():
        if modelTwo.getSpecies(one_species.getId()) is None:
            return False
    for one_reaction in modelOne.getListOfReactions():
        if modelTwo.getReaction(one_reaction.getId()) is None:
            return False
    for one_compartment in modelOne.getListOfCompartments():
        if modelTwo.getCompartment(one_compartment.getId()) is None:
            return False
    for one_parameter in modelOne.getListOfParameters():
        if modelTwo.getParameter(one_parameter.getId()) is None:
            return False
    for one_unitDef in modelOne.getListOfUnitDefinitions():
        if modelTwo.getUnitDefinition(one_unitDef.getId()) is None:
            return False
    return True

In [5]:
#createModel
rpsbml = rpSBML.rpSBML()
rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
libsbml.writeSBML(rpsbml.document, 'models/test_createModel.sbml')

1

In [6]:
#createUnitDefinition
rpsbml = rpSBML.rpSBML()
rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
unitDef = rpsbml.createUnitDefinition('mmol_per_gDW_per_hr')
libsbml.writeSBML(rpsbml.document, 'models/test_createUnitDefinition.sbml')

1

In [7]:
#createUnit
rpsbml = rpSBML.rpSBML()
rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
unitDef = rpsbml.createUnitDefinition('mmol_per_gDW_per_hr')
rpsbml.createUnit(unitDef, libsbml.UNIT_KIND_MOLE, 1, -3, 1)
rpsbml.createUnit(unitDef, libsbml.UNIT_KIND_GRAM, 1, 0, 1)
rpsbml.createUnit(unitDef, libsbml.UNIT_KIND_SECOND, 1, 0, 3600)
libsbml.writeSBML(rpsbml.document, 'models/test_createUnit.sbml')

1

In [8]:
#createParameter
rpsbml = rpSBML.rpSBML()
rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
upInfParam = rpsbml.createParameter('B_INF', float('inf'), 'kj_per_mol')
libsbml.writeSBML(rpsbml.document, 'models/test_createParameter.sbml')

1

In [9]:
#createCompartent
rpsbml = rpSBML.rpSBML()
rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
rpsbml.createCompartment(1, 'cytoplasm')
libsbml.writeSBML(rpsbml.document, 'models/test_createCompartent.sbml')

1

In [10]:
#genericModel
rpsbml = rpSBML.rpSBML()
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
libsbml.writeSBML(rpsbml.document, 'models/test_genericModel.sbml')

1

In [14]:
#createSpecies
rpsbml = rpSBML.rpSBML(None, None, None, None, '../cache')
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, None, inchi, smiles, 'cytoplasm', 0, '', None, None)
#libsbml.writeSBML(rpsbml.document, 'test_models/test_createSpecies.sbml')

In [13]:
inModel = libsbml.readSBML('test_models/test_createSpecies.sbml')

In [15]:
inModel.getModel().toXMLNode().equals(rpsbml.model.toXMLNode())

False

In [117]:
a = inModel.getModel()
b = rpsbml.model

In [141]:
a.species[0].getA

<Species TARGET_0000000001__64__cytoplasm "TARGET_0000000001">

In [140]:
f.setStrict()

TypeError: setStrict() missing 1 required positional argument: 'strict'

In [132]:
g = f[0]

In [133]:
g.getUnit()

NotImplementedError: Wrong number or type of arguments for overloaded function 'UnitDefinition_getUnit'.
  Possible C/C++ prototypes are:
    UnitDefinition::getUnit(unsigned int)
    UnitDefinition::getUnit(unsigned int) const


In [85]:
a.equals(b)

False

In [36]:
a.equals(a)

True

In [49]:
m = rpsbml.model

In [53]:
c = m.getSpecies(0)

In [54]:
c.toXMLNode()

<libsbml.XMLNode proxy of <Swig Object of type 'XMLNode_t *' at 0x7f89f23e68d0> >

In [55]:
rpsbml.model.getSpecies(0).toXMLNode().equals(inModel.getModel().getSpecies(0).toXMLNode())

False

In [78]:
d = rpsbml.model.getSpecies('MNXM13__64__cytoplasm')

In [79]:
d1 = inModel.getModel().getSpecies('MNXM13__64__cytoplasm')

In [105]:
f = rpsbml.model.getListOfSpecies()

In [106]:
g = f[0]

In [108]:
a.getListOfReactions

'MNXM13__64__cytoplasm'

In [115]:
a.getUnitDefinition

In [81]:
v1 = d1.toXMLNode()

In [86]:
v.equals(v1)

True

In [75]:
print(v.toXMLString())

<species metaid="_9e590de202b6a3a9b1c3566cafda86d0" id="MNXM13__64__cytoplasm" name="MNXM13" compartment="cytoplasm" initialConcentration="1" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false" fbc:charge="0">
  <annotation>
    <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
      <rdf:Description rdf:about="#_9e590de202b6a3a9b1c3566cafda86d0">
        <bqbiol:is>
          <rdf:Bag>
            <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/co2"/>
            <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/M_co2"/>
            <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM13"/>
            <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM89563"/>
            <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:16526"/>
            <rdf:li rdf:resource="http://identifi

In [69]:
print(v1.toXMLString())

<species metaid="_76aec9c4390657d33b71dbc0bef4908b" id="TARGET_0000000001__64__cytoplasm" name="TARGET_0000000001" compartment="cytoplasm" initialConcentration="1" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false" fbc:charge="0">
  <annotation>
    <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
      <rdf:Description rdf:about="#_76aec9c4390657d33b71dbc0bef4908b">
        <bqbiol:is>
          <rdf:Bag/>
        </bqbiol:is>
      </rdf:Description>
      <rdf:Ibisba rdf:about="#_76aec9c4390657d33b71dbc0bef4908b">
        <ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
          <ibisba:smiles>[H]OC(=O)C([H])=C([H])C([H])=C([H])C(=O)O[H]</ibisba:smiles>
          <ibisba:inchi>InChI=1S/C6H6O4/c7-5(8)3-1-2-4-6(9)10/h1-4H,(H,7,8)(H,9,10)</ibisba:inchi>
          <ibisba:inchikey>TXXHDPDFNKHHGW-UHFFFAOYSA-N</ibisba:inchikey>
  

In [28]:
print(libsbml.XMLNode.convertXMLNodeToString(inModel.toXMLNode()))

<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" level="3" version="1" groups:required="false" fbc:required="false">
  <model metaid="_81bcdb4f4f90440a19855f20c14ac474" id="rp_model" name="RetroPath_heterologous_pathway" substanceUnits="mole" timeUnits="second" extentUnits="mole">
    <listOfUnitDefinitions>
      <unitDefinition metaid="aebbce66773195cbd5d82838dda4f0ea" id="mmol_per_gDW_per_hr">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="gram" exponent="1" scale="0" multiplier="1"/>
          <unit kind="second" exponent="1" scale="0" multiplier="3600"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition metaid="_0346858f65275ff01aa2ac815127e134" id="kj_per_mol">
        <listOfUnits>
          <unit kind="joule" exponent="1" scale="3" multiplier="1"/>

In [29]:
print(libsbml.XMLNode.convertXMLNodeToString(rpsbml.document.toXMLNode()))

<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" level="3" version="1" groups:required="false" fbc:required="false">
  <model metaid="_81bcdb4f4f90440a19855f20c14ac474" id="rp_model" name="RetroPath_heterologous_pathway" substanceUnits="mole" timeUnits="second" extentUnits="mole">
    <listOfUnitDefinitions>
      <unitDefinition metaid="aebbce66773195cbd5d82838dda4f0ea" id="mmol_per_gDW_per_hr">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="gram" exponent="1" scale="0" multiplier="1"/>
          <unit kind="second" exponent="1" scale="0" multiplier="3600"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition metaid="_0346858f65275ff01aa2ac815127e134" id="kj_per_mol">
        <listOfUnits>
          <unit kind="joule" exponent="1" scale="3" multiplier="1"/>

In [16]:
print(inModel.toSBML())

<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" level="3" version="1" groups:required="false" fbc:required="false">
  <model metaid="_81bcdb4f4f90440a19855f20c14ac474" id="rp_model" name="RetroPath_heterologous_pathway" substanceUnits="mole" timeUnits="second" extentUnits="mole">
    <listOfUnitDefinitions>
      <unitDefinition metaid="aebbce66773195cbd5d82838dda4f0ea" id="mmol_per_gDW_per_hr">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="gram" exponent="1" scale="0" multiplier="1"/>
          <unit kind="second" exponent="1" scale="0" multiplier="3600"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition metaid="_0346858f65275ff01aa2ac815127e134" id="kj_per_mol">
        <listOfUnits>
          <unit kind="joule" exponent="1" scale="3" multiplier="1"/>

In [17]:
print(rpsbml.document.toSBML())

<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" level="3" version="1" groups:required="false" fbc:required="false">
  <model metaid="_81bcdb4f4f90440a19855f20c14ac474" id="rp_model" name="RetroPath_heterologous_pathway" substanceUnits="mole" timeUnits="second" extentUnits="mole">
    <listOfUnitDefinitions>
      <unitDefinition metaid="aebbce66773195cbd5d82838dda4f0ea" id="mmol_per_gDW_per_hr">
        <listOfUnits>
          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>
          <unit kind="gram" exponent="1" scale="0" multiplier="1"/>
          <unit kind="second" exponent="1" scale="0" multiplier="3600"/>
        </listOfUnits>
      </unitDefinition>
      <unitDefinition metaid="_0346858f65275ff01aa2ac815127e134" id="kj_per_mol">
        <listOfUnits>
          <unit kind="joule" exponent="1" scale="3" multiplier="1"/>

In [12]:
#createPathway
rpsbml = rpSBML.rpSBML()
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, None, inchi, smiles, 'cytoplasm', 0, '', None, None)
rp_pathway = rpsbml.createPathway(path_id)
#reactions
step_id = 0
for stepNum in range(len(steps)):
    rpsbml.createReaction('RP_'+str(stepNum), 'B_INF', 'B__INF', steps[stepNum], reaction_smiles[stepNum], 'cytoplasm', rpsbml.hetero_group, None)
    step_id += 1
libsbml.writeSBML(rpsbml.document, 'models/test_createPathway.sbml')

1

In [13]:
#createReaction
rpsbml = rpSBML.rpSBML()
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, None, inchi, smiles, 'cytoplasm', 0, '', None, None)
rpsbml.createPathway(path_id)
#reactions
step_id = 0
for stepNum in range(len(steps)):
    rpsbml.createReaction('RP_'+str(stepNum), 'B_INF', 'B__INF', steps[stepNum], reaction_smiles[stepNum], 'cytoplasm', None, None)
    step_id += 1
libsbml.writeSBML(rpsbml.document, 'models/test_createReaction.sbml')

1

In [14]:
#createFluxObj
rpsbml = rpSBML.rpSBML()
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, None, inchi, smiles, 'cytoplasm', 0, '', None, None)
rpsbml.createPathway(path_id)
step_id = 0
for stepNum in range(len(steps)):
    rpsbml.createReaction('RP_'+str(stepNum), 'B_INF', 'B__INF', steps[stepNum], reaction_smiles[stepNum], 'cytoplasm')
    step_id += 1
rpsbml.createFluxObj('flux1', 'RP_1', 2.0, True)
libsbml.writeSBML(rpsbml.document, 'models/test_createFluxObj.sbml')

1

In [15]:
#readAnnotation
rpsbml = rpSBML.rpSBML()
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, None, inchi, smiles, 'cytoplasm', 0, '', None, None)
libsbml.writeSBML(rpsbml.document, 'models/test_readAnnotation.sbml')

1

In [17]:
#compare annotation
#rpsbml = rpSBML.rpSBML()
#document = libsbml.readSBML('models/test_createPathway.sbml')
#model = document.getModel()

In [19]:
model.species[0].getAnnotation()

<libsbml.XMLNode proxy of <Swig Object of type 'XMLNode_t *' at 0x7f98be04c510> >

In [92]:
yeast_document = libsbml.readSBML('/home/mdulac/Downloads/bigg_iMM904.COBRA-sbml3.xml')
yeast_model = yeast_document.getModel()
rpsbml.mergeModels(yeast_document.getModel())

In [4]:
rpsbml.readAnnotation(rpsbml.model.getSpecies('MNXM1__64__cytoplasm').getAnnotation())

{'bigg.metabolite': ['h', 'M_h'],
 'metanetx.chemical': ['MNXM1', 'MNXM145872', 'MNXM89553'],
 'chebi': ['CHEBI:15378', 'CHEBI:10744', 'CHEBI:13357', 'CHEBI:5584'],
 'hmdb': ['HMDB59597'],
 'kegg.compound': ['C00080'],
 'seed.compound': ['cpd00067']}

In [5]:
a = rpsbml.model.getSpecies('MNXM1__64__cytoplasm').getAnnotation()

In [6]:
rpsbml.readIBISBAAnnotation(a)

{'smiles': '[H+]',
 'inchi': 'InChI=1S',
 'inchikey': '',
 'ddG': {'units': 'kj_per_mol', 'value': ''},
 'ddG_uncert': {'units': 'kj_per_mol', 'value': ''}}

In [12]:
b = a.getChild('RDF').getChild('Ibisba').getChild('ibisba')

In [13]:
b.getNumChildren()

5

In [10]:
print(b.toXMLString())

<ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
  <ibisba:smiles>[H+]</ibisba:smiles>
  <ibisba:inchi>InChI=1S</ibisba:inchi>
  <ibisba:inchikey/>
  <ibisba:ddG units="kj_per_mol"/>
  <ibisba:ddG_uncert units="kj_per_mol"/>
</ibisba:ibisba>


In [19]:
c = b.getChild(0)

In [22]:
c.toXMLString()

'<ibisba:smiles>[H+]</ibisba:smiles>'

In [24]:
c.getName()

'smiles'

In [34]:
b.getChild(0).getChild(0).toXMLString()

'[H+]'

In [35]:
print(b.toXMLString())

<ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
  <ibisba:smiles>[H+]</ibisba:smiles>
  <ibisba:inchi>InChI=1S</ibisba:inchi>
  <ibisba:inchikey/>
  <ibisba:ddG units="kj_per_mol"/>
  <ibisba:ddG_uncert units="kj_per_mol"/>
</ibisba:ibisba>


In [33]:
b = a.getChild('RDF').getChild('Ibisba')

In [34]:
print(b.toXMLString())

<rdf:Ibisba rdf:about="#_8abbc7f30aac6ab1c526b763e164dbc5">
  <ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
    <ibisba:smiles>[H+]</ibisba:smiles>
    <ibisba:inchi>InChI=1S</ibisba:inchi>
    <ibisba:inchikey/>
    <ibisba:parameter type="ddG" units="kj_per_mol"/>
    <ibisba:parameter type="ddG_uncert" units="kj_per_mol"/>
  </ibisba:ibisba>
</rdf:Ibisba>


In [35]:
c = b.getChild('ibisba')

In [36]:
print(c.toXMLString())

<ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
  <ibisba:smiles>[H+]</ibisba:smiles>
  <ibisba:inchi>InChI=1S</ibisba:inchi>
  <ibisba:inchikey/>
  <ibisba:parameter type="ddG" units="kj_per_mol"/>
  <ibisba:parameter type="ddG_uncert" units="kj_per_mol"/>
</ibisba:ibisba>


In [67]:
c.getChild(3).toXMLString()

'<ibisba:parameter type="ddG" units="kj_per_mol"/>'

In [68]:
d = c.getChild(3)

In [73]:
d.getAttrValue('value')

''

In [37]:
d = c.getChild(0)

In [38]:
d.toXMLString()

'<ibisba:smiles>[H+]</ibisba:smiles>'

In [58]:
d.getName()

'smiles'

In [61]:
d.getChild(0).toXMLString()

'[H+]'

In [62]:
d.getPrefix()

'ibisba'

In [63]:
d.getURI()

'http://ibisba.eu/qualifiers'

In [65]:
d.getName

10

In [35]:
a.getIndex('RDF')

0

In [37]:
b = a.getChild('RDF')

In [38]:
b.getIndex('Description')

0

In [30]:
print(b.toXMLString())

<rdf:Description rdf:about="#ee08cafb5274b6989e332af7c06ce4f6">
  <bqbiol:is>
    <rdf:Bag>
      <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/o2"/>
      <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/M_o2"/>
      <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM4"/>
      <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM114217"/>
      <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM89559"/>
      <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM97007"/>
      <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15379"/>
      <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:10745"/>
      <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13416"/>
      <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:23833"/>
      <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:25366"/>
      <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:29097"/>
   

In [26]:
print(a.toXMLString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#ee08cafb5274b6989e332af7c06ce4f6">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/o2"/>
          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/M_o2"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM4"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM114217"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM89559"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM97007"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15379"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:10745"/>
          <rdf:li rdf:resource="http://identifiers.org/

In [15]:
a = rpsbml.model.species[6].getAnnotation()

In [16]:
print(a.toXMLString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#_4354ed2227b0469132f44f938fb566fc">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/nad"/>
          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/M_nad"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM8"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM588016"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM96411"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:15846"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13394"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:21901"/>
          <rdf:li rdf:resource="http://identifiers.org/chebi/C

In [45]:
rpsbml = rpSBML.rpSBML()
######### Model
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
#list all the unique metabolites that must be created
#identify if there are InchI or SMILES description of the two different molecules
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, None, inchi, smiles, 'cytoplasm')
rpsbml.createPathway(path_id)
#reactions
step_id = 0
for stepNum in range(len(steps)):
    rpsbml.createReaction('RP_'+str(stepNum), 'B_INF', 'B__INF', steps[stepNum], reaction_smiles[stepNum], 'cytoplasm')
    step_id += 1

In [82]:
rpsbml.chemXref['MNXM237']

{'bigg': ['10fthf', 'M_10fthf'],
 'chebi': ['15637', '11304', '19108', '19109', '698', '57454'],
 'hmdb': ['HMDB00972'],
 'kegg': ['C00234'],
 'metacyc': ['10-FORMYL-THF'],
 'reactome': ['419151', '5389850'],
 'sabiork': ['1318'],
 'seed': ['cpd00201']}

In [46]:
rpsbml.createFluxObj('flux1', 'RP_1', 2.0, True)

In [40]:
rpsbml = rpSBML.rpSBML()

rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
# mmol_per_gDW_per_hr

unitDef = rpsbml.createUnitDefinition('mmol_per_gDW_per_hr')
rpsbml.createUnit(unitDef, libsbml.UNIT_KIND_MOLE, 1, -3, 1)
rpsbml.createUnit(unitDef, libsbml.UNIT_KIND_GRAM, 1, 0, 1)
rpsbml.createUnit(unitDef, libsbml.UNIT_KIND_SECOND, 1, 0, 3600)
# kj_per_mol
gibbsDef = rpsbml.createUnitDefinition('kj_per_mol')
rpsbml.createUnit(gibbsDef, libsbml.UNIT_KIND_JOULE, 1, 3, 1)
rpsbml.createUnit(gibbsDef, libsbml.UNIT_KIND_MOLE, 1, 1, 1)
# infinity parameters (FBA)

upInfParam = rpsbml.createParameter('B_INF', float('inf'), 'kj_per_mol')
lowInfParam = rpsbml.createParameter('B__INF', float('-inf'), 'kj_per_mol')

#compartment
rpsbml.createCompartment(1, 'cytoplasm')


"\nunitDef = rpsbml.createUnitDefinition('mmol_per_gDW_per_hr')\nrpsbml.createUnit(unitDef, libsbml.UNIT_KIND_MOLE, 1, -3, 1)\nrpsbml.createUnit(unitDef, libsbml.UNIT_KIND_GRAM, 1, 0, 1)\nrpsbml.createUnit(unitDef, libsbml.UNIT_KIND_SECOND, 1, 0, 3600)\n# kj_per_mol\ngibbsDef = rpsbml.createUnitDefinition('kj_per_mol')\nrpsbml.createUnit(gibbsDef, libsbml.UNIT_KIND_JOULE, 1, 3, 1)\nrpsbml.createUnit(gibbsDef, libsbml.UNIT_KIND_MOLE, 1, 1, 1)\n# infinity parameters (FBA)\n\nupInfParam = rpsbml.createParameter('B_INF', float('inf'), 'kj_per_mol')\nlowInfParam = rpsbml.createParameter('B__INF', float('-inf'), 'kj_per_mol')\n"

In [ ]:
rpsbml = rpSBML.rpSBML()
rpsbml.createModel('RetroPath_heterologous_pathway', 'rp_model')
rpsbml.createFluxObj(1, )

In [57]:
rpsbml._genMetaID('###test_input-23####')

'_5df40e51a5d358ecfdf0372317853a79'

In [70]:
rpsbml.readAnnotation(rpsbml.model.species[3].getAnnotation())

{}

In [79]:
a = rpsbml.model.species[8]

In [80]:
print(a.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#_9e590de202b6a3a9b1c3566cafda86d0">
      <bqbiol:is>
        <rdf:Bag/>
      </bqbiol:is>
    </rdf:Description>
    <rdf:Description rdf:about="#_9e590de202b6a3a9b1c3566cafda86d0">
      <ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
        <ibisba:smiles>O=C=O</ibisba:smiles>
        <ibisba:inchi>InChI=1S/CO2/c2-1-3</ibisba:inchi>
        <ibisba:inchikey>CURLTUGMZLYLDI-UHFFFAOYSA-N</ibisba:inchikey>
        <ibisba:parameter type="ddG" units="kj_per_mol"/>
        <ibisba:parameter type="ddG_uncert" units="kj_per_mol"/>
      </ibisba:ibisba>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [ ]:
rpsbml._nameToSbmlId('test_input')

In [47]:
#rpsbml = rpSBML.rpSBML()
#rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
#pickle.dump({'rpsbml': rpsbml.model}, open('test/rpsbml.pickle', 'wb'))
libsbml.writeSBML(rpsbml.document, 'test/models/test_createFluxObj.sbml')

1

In [11]:
inModel = libsbml.readSBML('test/models/test_createSpecies.sbml')

In [23]:
rpsbml.document.toSBML()

'<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" level="3" version="1" groups:required="false" fbc:required="false">\n  <model metaid="_81bcdb4f4f90440a19855f20c14ac474" id="rp_model" name="RetroPath_heterologous_pathway" substanceUnits="mole" timeUnits="second" extentUnits="mole">\n    <listOfUnitDefinitions>\n      <unitDefinition metaid="aebbce66773195cbd5d82838dda4f0ea" id="mmol_per_gDW_per_hr">\n        <listOfUnits>\n          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>\n          <unit kind="gram" exponent="1" scale="0" multiplier="1"/>\n          <unit kind="second" exponent="1" scale="0" multiplier="3600"/>\n        </listOfUnits>\n      </unitDefinition>\n      <unitDefinition metaid="_0346858f65275ff01aa2ac815127e134" id="kj_per_mol">\n        <listOfUnits>\n          <unit kind="joule" exponent="1" scale="3" mul

In [21]:
inModel.toSBML()

'<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" level="3" version="1" groups:required="false" fbc:required="false">\n  <model metaid="_81bcdb4f4f90440a19855f20c14ac474" id="rp_model" name="RetroPath_heterologous_pathway" substanceUnits="mole" timeUnits="second" extentUnits="mole">\n    <listOfUnitDefinitions>\n      <unitDefinition metaid="aebbce66773195cbd5d82838dda4f0ea" id="mmol_per_gDW_per_hr">\n        <listOfUnits>\n          <unit kind="mole" exponent="1" scale="-3" multiplier="1"/>\n          <unit kind="gram" exponent="1" scale="0" multiplier="1"/>\n          <unit kind="second" exponent="1" scale="0" multiplier="3600"/>\n        </listOfUnits>\n      </unitDefinition>\n      <unitDefinition metaid="_0346858f65275ff01aa2ac815127e134" id="kj_per_mol">\n        <listOfUnits>\n          <unit kind="joule" exponent="1" scale="3" mul

In [9]:
libsbml.writeSBML(rpsbml.document, 'test_create.sbml')

1

In [10]:
yeast_document = libsbml.readSBML('/home/mdulac/Downloads/bigg_iMM904.COBRA-sbml3.xml')
rpsbml.mergeModels(yeast_document.getModel())

In [11]:
libsbml.writeSBML(yeast_document, 'test_merge.sbml')

1

In [12]:
rpsbml._genMetaID('hjsdhdjs')

'_0af541538e84731aa9bd5c492976b25c'

In [ ]:
#### generate the xref pickles ####

In [9]:
chemXref = {}
with open('/home/mdulac/Downloads/chem_xref.tsv') as f:
    c = csv.reader(f, delimiter='\t')
    for row in c:
        if not row[0][0]=='#' and len(row[0].split(':'))==2 and not row[0].split(':')[0]=='deprecated':
            mnx = row[1]
            dbName = row[0].split(':')[0]
            dbID = row[0].split(':')[1]
            if not mnx in chemXref:
                chemXref[mnx] = {}
            if not dbName in chemXref[mnx]:
                chemXref[mnx][dbName] = []
            chemXref[mnx][dbName].append(dbID)
pickle.dump(chemXref, open('/home/mdulac/workspace/rpFBA/cache/chemXref.pickle', 'wb'))

In [11]:
chemXref

{'MNXM237': {'bigg': ['10fthf', 'M_10fthf'],
  'chebi': ['15637', '11304', '19108', '19109', '698', '57454'],
  'hmdb': ['HMDB00972'],
  'kegg': ['C00234'],
  'metacyc': ['10-FORMYL-THF'],
  'reactome': ['419151', '5389850'],
  'sabiork': ['1318'],
  'seed': ['cpd00201']},
 'MNXM3428': {'bigg': ['10fthf5glu', 'M_10fthf5glu']},
 'MNXM3429': {'bigg': ['10fthf6glu', 'M_10fthf6glu']},
 'MNXM5422': {'bigg': ['10fthf7glu', 'M_10fthf7glu']},
 'MNXM164047': {'bigg': ['10fthfglu__L', 'M_10fthfglu__L'],
  'chebi': ['27862', '19110', '699']},
 'MNXM146564': {'bigg': ['11_cis_retfa', 'M_11_cis_retfa']},
 'MNXM163103': {'bigg': ['11docrtsl', 'M_11docrtsl']},
 'MNXM162887': {'bigg': ['11docrtstrn', 'M_11docrtstrn']},
 'MNXM13989': {'bigg': ['12HPET', 'M_12HPET']},
 'MNXM13994': {'bigg': ['12RHPET', 'M_12RHPET']},
 'MNXM163631': {'bigg': ['12d3k5m', 'M_12d3k5m']},
 'MNXM163072': {'bigg': ['12dag3p_BS', 'M_12dag3p_BS']},
 'MNXM4939': {'bigg': ['12dgr120', 'M_12dgr120']},
 'MNXM146479': {'bigg': ['12dg

In [10]:
reacXref = {}
with open('/home/mdulac/Downloads/reac_xref.tsv') as f:
    c = csv.reader(f, delimiter='\t')
    for row in c:
        if not row[0][0]=='#' and len(row[0].split(':'))==2:
            mnx = row[1]
            dbName = row[0].split(':')[0]
            dbID = row[0].split(':')[1]
            if not mnx in reacXref:
                reacXref[mnx] = {}
            if not dbName in reacXref[mnx]:
                reacXref[mnx][dbName] = []
            reacXref[mnx][dbName].append(dbID)
pickle.dump(reacXref, open('/home/mdulac/workspace/rpFBA/cache/reacXref.pickle', 'wb'))

In [11]:
mnxc_name = {}
with open('/home/mdulac/Downloads/comp_prop.tsv') as f:
    c = csv.reader(f, delimiter='\t')
    for row in c:
        if not row[0][0]=='#':
            mnxc = row[0]
            name = row[1]
            if not mnxc in mnxc_name:
                mnxc_name[mnxc] = name
compXref = {}
with open('/home/mdulac/Downloads/comp_xref.tsv') as f:
    c = csv.reader(f, delimiter='\t')
    for row in c:
        if not row[0][0]=='#' and len(row[0].split(':'))==2:
            mnxc = row[1]
            name = mnxc_name[mnxc]
            dbName = row[0].split(':')[0]
            dbID = row[0].split(':')[1]
            if not name in compXref:
                compXref[name] = {}
            if not dbName in compXref[name]:
                compXref[name][dbName] = []
            if not 'mnx' in compXref[name]:
                compXref[name]['mnx'] = []
            if not mnxc in compXref[name]['mnx']:
                compXref[name]['mnx'].append(mnxc)
            compXref[name][dbName].append(dbID)
pickle.dump(compXref, open('/home/mdulac/workspace/rpFBA/cache/compXref.pickle', 'wb'))

In [ ]:
compXref['cytoplasm']['seed']

In [6]:
unit_def = rp_model.getListOfUnitDefinitions()

In [12]:
a = 'MNXC17'

In [14]:
a[:4]

'MNXC'

In [8]:
rp_model.getNumUnitDefinitions()

1

In [18]:
[i.getId() for i in rp_model.getListOfUnitDefinitions()]

['mmol_per_gDW_per_hr']

In [12]:
checklibSBML(yeast_model.addUnitDefinition(unit_def[0]), 'adding unit def')

Error encountered trying to adding unit def.LibSBML returned error code -10: "The SBML Namespaces associated with the object do not match the SBML Namespaces of the parent object. This error can happen when an SBML component such as a species or compartment object is created outside of a model and a calling program then attempts to add the object to a model that has a different SBML Namespaces combination."


SystemExit: Error encountered trying to adding unit def.LibSBML returned error code -10: "The SBML Namespaces associated with the object do not match the SBML Namespaces of the parent object. This error can happen when an SBML component such as a species or compartment object is created outside of a model and a calling program then attempts to add the object to a model that has a different SBML Namespaces combination."

/home/mdulac/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
unit_def = rp_model.getListOfUnitDefinitions()

In [18]:
a = unit_def[0]

In [21]:
a.getListOfUnits()

<ListOfUnits[3]>

In [25]:
for i in unit_def:
    print(i)

<UnitDefinition mmol_per_gDW_per_hr>


In [30]:
u = unit_def[0].getUnit(0)

In [39]:
[i.getId() for i in rp_model.getListOfCompartments()]

['MNXC3']

In [34]:
target_unitDef = yeast_model.createUnitDefinition()

In [42]:
a = yeast_model.getListOfCompartments()[1].getAnnotation()

In [9]:
spe = yeast_model.getReaction(1)

In [10]:
yeast_fbc = spe.getPlugin('fbc')

In [14]:
yeast_fbc.getLowerFluxBound()

'B__999999'

In [63]:
a = spe.getReactant(0)

In [64]:
a.species

'MNXM01__64__MNXC2'

In [60]:
reactant = spe.getProduct

In [61]:
reactant.setS

<SpeciesReference>

In [72]:
yeast_fbc = yeast_model.getPlugin('fbc')

In [2]:
a = [1,2,3]

In [4]:
if not 5 in a:
    print('T')

T


In [78]:
yeast_fbc.getObjective(0)

<Objective obj_RB54E0D8C>

In [75]:
yeast_fbc.getListOfGeneProducts

1

In [118]:
a = yeast_fbc.getListOfObjectives()

In [120]:
a[0]

<Objective obj_RB54E0D8C>

In [121]:
b = a[0]

In [132]:
c = b.getFluxObjective(0)

In [135]:
c.getReaction()

'RB54E0D8C'

<bound method Objective.getFluxObjective of <Objective obj_RB54E0D8C>>

In [86]:
a = yeast_fbc.createGeneProduct()

In [ ]:
yeast_fbc.getGeneProduct

In [136]:
yeast_model.getListOfParameters()

<ListOfParameters[6]>

In [137]:
a = yeast_model.getListOfParameters()[0]

In [139]:
a.getConstant()

True

In [79]:
yeast_fbc.addObjective()

TypeError: addObjective() missing 1 required positional argument: 'o'

In [68]:
a = yeast_fbc.getListOfGeneProducts()

In [ ]:
a.

In [70]:
b = a[0]

In [71]:
b.getId()

'G_AGP1_YEAST'

In [ ]:
yeast_fbc.getListOfFluxBounds

In [6]:
path_id = 1
steps = [{'right': {'CMPD_0000000003': 1, 'MNXM13': 1, 'MNXM15': 1, 'MNXM8': 1}, 'left': {'MNXM10': 1, 'MNXM188': 1, 'MNXM4': 1, 'MNXM1': 3}}, {'right': {'TARGET_0000000001': 1, 'MNXM1': 2}, 'left': {'CMPD_0000000003': 1, 'MNXM4': 1, }}]
reaction_smiles = ['[H]Oc1c([H])c([H])c([H])c([H])c1O[H]>>O=O.[H]N=C(O[H])C1=C([H])N(C2([H])OC([H])(C([H])([H])OP(=O)(O[H])OP(=O)(O[H])OC([H])([H])C3([H])OC([H])(n4c([H])nc5c(N([H])[H])nc([H])nc54)C([H])(O[H])C3([H])O[H])C([H])(O[H])C2([H])O[H])C([H])=C([H])C1([H])[H].[H]OC(=O)c1c([H])c([H])c([H])c([H])c1N([H])[H]', '[H]OC(=O)C([H])=C([H])C([H])=C([H])C(=O)O[H]>>O=O.[H]Oc1c([H])c([H])c([H])c([H])c1O[H]', ]
rp_smiles = {'MNXM10': '[H]N=C(O[H])C1=C([H])N(C2([H])OC([H])(C([H])([H])OP(=O)(O[H])OP(=O)(O[H])OC([H])([H])C3([H])OC([H])(n4c([H])nc5c(N([H])[H])nc([H])nc54)C([H])(O[H])C3([H])O[H])C([H])(O[H])C2([H])O[H])C([H])=C([H])C1([H])[H]',
'MNXM188': '[H]OC(=O)c1c([H])c([H])c([H])c([H])c1N([H])[H]',
'MNXM4': 'O=O',
'MNXM1': '[H+]',
'CMPD_0000000003': '[H]Oc1c([H])c([H])c([H])c([H])c1O[H]',
'MNXM13': 'O=C=O',
'MNXM15': '[H]N([H])[H]',
'MNXM8': 'NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)OP([O-])(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O',
'TARGET_0000000001': '[H]OC(=O)C([H])=C([H])C([H])=C([H])C(=O)O[H]'}
rp_inchi = {'MNXM10': 'InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1,3-4,7-8,10-11,13-16,20-21,29-32H,2,5-6H2,(H2,23,33)(H,34,35)(H,36,37)(H2,22,24,25)/p-2/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1',
'MNXM188': 'InChI=1S/C7H7NO2/c8-6-4-2-1-3-5(6)7(9)10/h1-4H,8H2,(H,9,10)/p-1',
'MNXM4': 'InChI=1S/O2/c1-2',
'MNXM1': 'InChI=1S',
'CMPD_0000000003': None,
'MNXM13': 'InChI=1S/CO2/c2-1-3',
'MNXM15': 'InChI=1S/H3N/h1H3/p+1',
'MNXM8': 'InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p-1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1',
'TARGET_0000000001': 'InChI=1S/C6H6O4/c7-5(8)3-1-2-4-6(9)10/h1-4H,(H,7,8)(H,9,10)'}

In [7]:
rpsbml = rpSBML.rpSBML()
######### Model
rpsbml.genericModel('RetroPath_heterologous_pathway', 'rp_model')
#list all the unique metabolites that must be created
#identify if there are InchI or SMILES description of the two different molecules
for meta in set([i for step in steps for lr in ['left', 'right'] for i in step[lr]]):
    try:
        inchi = rp_inchi[meta]
    except KeyError:
        inchi = None
    try:
        smiles = rp_smiles[meta]
    except KeyError:
        smiles = None
    rpsbml.createSpecies(meta, {}, None, inchi, smiles)
rpsbml.createPathway(path_id)
#reactions
step_id = 0
for stepNum in range(len(steps)):
    rpsbml.createReaction('RP_'+str(stepNum), 'RP_id'+str(stepNum), 'B_INF', 'B__INF', steps[stepNum], reaction_smiles[stepNum])
    step_id += 1

In [8]:
yeast_document = libsbml.readSBML('/home/mdulac/Downloads/bigg_iMM904.COBRA-sbml3.xml')
rpsbml.mergeModels(yeast_document.getModel())

In [9]:
yeast_model = yeast_document.getModel()

In [10]:
yeast_model.getNumSpecies()

1240

In [11]:
yeast_model_ori.getNumSpecies()

1231

In [12]:
libsbml.writeSBML(yeast_document, 'test_merge.sbml')

1

In [9]:
rpsbml.model.getNumSpecies()

9

In [9]:
a = rpsbml.model.species[3].getAnnotation()

In [11]:
a.toXMLString()

'<annotation>\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:ibisba="http://ibisba.eu/qualifiers">\n    <rdf:Description rdf:about="#_4354ed2227b0469132f44f938fb566fc">\n      <bqbiol:is>\n        <rdf:Bag/>\n      </bqbiol:is>\n    </rdf:Description>\n    <rdf:Description rdf:about="#_4354ed2227b0469132f44f938fb566fc">\n      <ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">\n        <ibisba:smiles>NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)OP([O-])(=O)OC[C@H]2O[C@H]([C@H](O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O</ibisba:smiles>\n        <ibisba:inchi>InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17)28(8-26-12)21-16(32)14(30)11(41-21)6-39-44(36,37)42-43(34,35)38-5-10-13(29)15(31)20(40-10)27-3-1-2-9(4-27)18(23)33/h1-4,7-8,10-11,13-16,20-21,29-32H,5-6H2,(H5-,22,23,24,25,33,34,35,36,37)/p-1/t10-,11-,13-,14-,15-,16-,20-,21-/m1/s1</ibisba:i

9
1231


AttributeError: type object 'object' has no attribute '__getattr__'

In [11]:
rp_model.species[1].getAnnotation()

<libsbml.XMLNode proxy of <Swig Object of type 'XMLNode_t *' at 0x7f561d36c660> >

In [14]:
rpsbml.model.getNumSpecies()

AttributeError: type object 'object' has no attribute '__getattr__'

In [10]:
len(steps[0])+len(steps[1])

4

In [11]:
a = set([i for step in steps for lr in ['left', 'right'] for i in step[lr]])

In [12]:
len(a)

9

In [13]:
a

{'CMPD_0000000003',
 'MNXM1',
 'MNXM10',
 'MNXM13',
 'MNXM15',
 'MNXM188',
 'MNXM4',
 'MNXM8',
 'TARGET_0000000001'}

In [38]:
a = rp_model.species[1]

In [ ]:
target_mo

In [49]:
print(a.toSBML())

<species metaid="_d494ab9e82db1edaa73b2eb7df7d767c" id="MNXM15__64__MNXC3" compartment="MNXC3" initialConcentration="1" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false">
  <annotation>
    <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:ibisba="http://ibisba.eu/qualifiers">
      <rdf:Description rdf:about="#_d494ab9e82db1edaa73b2eb7df7d767c">
        <bqbiol:is>
          <rdf:Bag>
            <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM15"/>
          </rdf:Bag>
        </bqbiol:is>
      </rdf:Description>
      <rdf:Description rdf:about="#_d494ab9e82db1edaa73b2eb7df7d767c">
        <ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
          <ibisba:smiles>[H]N([H])[H]</ibisba:smiles>
          <ibisba:inchi>InChI=1S/H3N/h1H3/p+1</ibisba:inchi>
          <ibisba:inchikey>QGZKDVFQNNGYKY-UHFFFAOYS

In [40]:
yeast_model.addSpecies(rp_model.species[1])

-10

In [44]:
yeast_species = yeast_model.createSpecies()

In [51]:
a.getAnnotation()

<libsbml.XMLNode proxy of <Swig Object of type 'XMLNode_t *' at 0x7f78369c1540> >

In [36]:
yeast2_sbmlns = libsbml.SBMLNamespaces(3,1)
yeast2_sbmlns.addPkgNamespace('groups', 1)
yeast2_sbmlns.addPkgNamespace('fbc', 2)
yeast2_document = libsbml.SBMLDocument(yeast2_sbmlns)
yeast2_document.setPackageRequired('fbc', False)
yeast2_document.setPackageRequired('groups', False)
yeast2_document.setModel(yeast_model.clone())
yeast2_model = yeast2_document.getModel()
#yeast2_model = yeast2_document.createModel()

In [37]:
yeast_sbmlns = yeast_document.getSBMLNamespaces()
rp_sbmlns = rp_document.getSBMLNamespaces()

In [7]:
yeast_model.enablePackage('http://www.sbml.org/sbml/level3/version1/groups/version1', 'groups', True)

0

In [6]:
yeast_model.isPackageEnabled('fbc')

True

In [11]:
yeast_groups = yeast_model.getPlugin('groups')

In [12]:
rp_groups = rp_model.getPlugin('groups')

In [17]:
a = rp_groups.getGroup('rp_pathway')

In [20]:
yeast_groups.addGroup(a)

0

In [38]:
yeast2_model.getPlugin('fbc')

<libsbml.FbcModelPlugin proxy of <Swig Object of type 'FbcModelPlugin *' at 0x7eff1b7b4660> >

In [40]:
y = yeast_model.clone()

In [50]:
yeast2_model.isPackageEnabled('groups')

True

In [49]:
yeast_model.enablePackage('http://www.sbml.org/sbml/level3/version1/groups/version1', 'groups', True)

0

In [51]:
yeast_model.enablePackage('http://www.sbml.org/sbml/level3/version1/groups/version1', 'groups', True)

0

In [ ]:
yeast_model.

In [21]:
libsbml.writeSBMLToFile(yeast_document, 'test_yeast_rp.sbml')

1

In [12]:
rp_fbc = rp_model.getPlugin('groups')

In [30]:
yeast2_document.meta_id

<Model>

In [21]:
yeast2_groups = yeast2_model.getPlugin('groups')

In [22]:
yeast2_groups

In [26]:
yeast2_document.isSetPkgRequired('fbc')

True

In [27]:
yeast2_model.

<Model bigg_iMM904 "Saccharomyces cerevisiae S288c">

In [28]:
yeast_model

<Model bigg_iMM904 "Saccharomyces cerevisiae S288c">

In [ ]:
yeast_model.getSBMLNamespaces()

In [19]:
type(yeast_model)

libsbml.Model

In [10]:
reac = rp_model.reactions[1]
reac_fbc = reac.getPlugin('fbc')

In [11]:
yeast_reac = yeast_model.reactions[101]
yeast_reac_fbc = yeast_reac.getPlugin('fbc')

In [67]:
rp_sbmlns.getNamespaces()

<libsbml.XMLNamespaces proxy of <Swig Object of type 'XMLNamespaces *' at 0x7f083d8f6960> >

In [72]:
b = rp_sbmlns.getNamespaces()

In [74]:
b.getNumNamespaces()

3

In [68]:
a = yeast_sbmlns.getNamespaces()

In [63]:
a.getURI()

'http://www.sbml.org/sbml/level3/version1/core'

In [53]:
yeast_document.enablePackage('groups',1, True)

ValueError: invalid null reference in method 'SBase_enablePackage', argument 3 of type 'std::string const &'

In [20]:
yeast_document.setPackageRequired('fbc', True)

0

In [36]:
checklibSBML(yeast_document.setPackageRequired('groups', False), 'setting groups required to false')

Error encountered trying to setting groups required to false.LibSBML returned error code -22: "The required version of the package extension is unknown. This error is typically returned when creating an object of SBase derived class with the required package, creating an object of SBMLNamespaces or its derived class with the required package, or invoking functions depending on the required package. This error may be avoided by updating the library of the required package to be linked."


SystemExit: Error encountered trying to setting groups required to false.LibSBML returned error code -22: "The required version of the package extension is unknown. This error is typically returned when creating an object of SBase derived class with the required package, creating an object of SBMLNamespaces or its derived class with the required package, or invoking functions depending on the required package. This error may be avoided by updating the library of the required package to be linked."

/home/mdulac/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [30]:
yeast_groups = yeast_model.getPlugin('groups')

In [31]:
yeast_groups

In [122]:
yeast_reac.getNumReactants()

2

In [123]:
yeast_reac.getNumProducts()

1

In [133]:
a = yeast_reac.reactants[0]

In [14]:
rp_groups = rp_model.getPlugin('groups')

In [15]:
rp_groups

<libsbml.GroupsModelPlugin proxy of <Swig Object of type 'GroupsModelPlugin *' at 0x7f083d8f63f0> >

In [160]:
print(a.toXMLString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#_72e6f6e0f08ca88f02b1480464afd55b">
      <ibisba:ibisba xmlns:ibisba="http://ibisba.eu">
        <ibisba:smiles>[H]OC(=O)C([H])=C([H])C([H])=C([H])C(=O)O[H]&gt;&gt;O=O.[H]Oc1c([H])c([H])c([H])c([H])c1O[H]</ibisba:smiles>
        <ibisba:parameter type="ddG" units="kj_per_mol" value="None"/>
        <ibisba:parameter type="ddG_uncert" units="kj_per_mol" value="None"/>
        <ibisba:fba_target/>
        <ibisba:fba_biomass/>
        <ibisba:fba_biLevel/>
      </ibisba:ibisba>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [143]:
a = {'one': 1}

In [144]:
a['tao']

KeyError: 'tao'

In [61]:
y = yeast_model.getElementBySId('MNXM8__64__MNXC3')

In [63]:
y.compartment

'MNXC3'

In [52]:
reac.getId()

'RP1'

In [69]:
reac_fbc.c

'_ec308451c1d095c528cfa3c009ea7235'

In [38]:
rp_model.getNumSpecies()

9

In [39]:
rp_model.species[1].getId()

'MNXM15__64__MNXC3'

In [54]:
for i in range(2):
    print('############')
    found = False
    for y in range(10):
        if y==5:
            found = True
            break
        print(str(i)+': '+str(y))
    if not found:
        print('##### '+str(i)+' #####')

############
0: 0
0: 1
0: 2
0: 3
0: 4
############
1: 0
1: 1
1: 2
1: 3
1: 4


In [35]:
print(rp_model.species[1].getAnnotation().getChild('RDF').getChild('Description').getChild('is').getChild('Bag').toXMLString())
print('###########')
print(yeast_model.species[101].getAnnotation().getChild('RDF').getChild('Description').getChild('is').getChild('Bag').toXMLString())

<rdf:Bag>
  <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM15"/>
</rdf:Bag>
###########
<rdf:Bag>
  <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM15"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13405"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13406"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13407"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13771"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:16134"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:22533"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:22534"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:28938"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:29337"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:29340"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:44269"/>
  <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:

In [ ]:
rpsbml.compareAnnotations(rp_model.species[1].getAnnotation(),
                       yeast_model.species[101].getAnnotation()
                      )

In [8]:
rpsbml = rpSBML.rpSBML()

True

In [123]:
yeast_model.species[0].getAnnotation()

<libsbml.XMLNode proxy of <Swig Object of type 'XMLNode_t *' at 0x7f671ec2e5a0> >

In [7]:
rp_model.reactions

<ListOfReactions[2]>

In [38]:
a = rp_model.species[0]

In [39]:
b = a.getAnnotation()

In [33]:
c = b.getAttributes()

In [46]:
print(b.toXMLString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:ibisba="http://ibisba.eu/qualifiers">
    <rdf:Description rdf:about="#_9dffa79758165386f3917928d2c97887">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM1"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
    <rdf:Description rdf:about="#_9dffa79758165386f3917928d2c97887">
      <ibisba:ibisba xmlns:ibisba="http://ibisba.eu/qualifiers">
        <ibisba:smiles>[H+]</ibisba:smiles>
        <ibisba:inchi>InChI=1S</ibisba:inchi>
        <ibisba:inchikey/>
        <ibisba:parameter type="ddG" units="kj_per_mol" value="None"/>
        <ibisba:parameter type="ddG_uncert" units="kj_per_mol" value="None"/>
      </ibisba:ibisba>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [45]:
b.hasChild('RDF')

True

In [49]:
c = b.getChild('RDF')
d = c.getChild('Description')
e = d.getChild('is')
f = e.getChild('Bag')

In [50]:
f.toXMLString()

'<rdf:Bag>\n  <rdf:li rdf:resource="http://identifiers.org/metanetx.chemical/MNXM1"/>\n</rdf:Bag>'

In [51]:
a = yeast_model.species[0]
b = a.getAnnotation()
c = b.getChild('RDF')
d = c.getChild('Description')
e = d.getChild('is')
f = e.getChild('Bag')

In [76]:
f = b.getChild('RDF').getChild('Description').getChild('is').getChild('Bag')

In [78]:
f.getNumChildren()

18

In [87]:
f.getChild(1)

''

In [88]:
g = f.getChild(1)

In [99]:
g.getNumChildren()

0

In [102]:
e = g.getAttrValue(0)

In [110]:
d = {'as': 1}

In [108]:
if d:
    print('T')

In [111]:
bool(d)

True

In [104]:
for i in range(f.getNumChildren()):
    print(f.getChild(i).getAttrValue(0).split('/')[-2])
    print(f.getChild(i).getAttrValue(0).split('/')[-1])
    print('################################')

metanetx.chemical
MNXM8
################################
chebi
CHEBI:13393
################################
chebi
CHEBI:13394
################################
chebi
CHEBI:15846
################################
chebi
CHEBI:21901
################################
chebi
CHEBI:29867
################################
chebi
CHEBI:44214
################################
chebi
CHEBI:44215
################################
chebi
CHEBI:44281
################################
chebi
CHEBI:57540
################################
chebi
CHEBI:7422
################################
bigg.metabolite
M_nad
################################
bigg.metabolite
nad
################################
hmdb
HMDB00902
################################
kegg.compound
C00003
################################
kegg.drug
D00002
################################
biocyc
META:NAD
################################
seed.compound
cpd00003
################################


In [71]:
g.toXMLString()

'<rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:13393"/>'

In [66]:
g.

1104

In [19]:
yeast_model.species.getId

<bound method SBase.getId of <ListOfSpecies[1231]>>

In [20]:
yeast_model = yeast_document.getModel()
yeast_model_ori = yeast_document_ori.getModel()
rp_model = rp_document.getModel()

In [16]:
yeast_model.appendFrom(rp_model)

0

In [11]:
yeast_model.species

<ListOfSpecies[1231]>

In [13]:
rp_model.species

<ListOfSpecies[9]>

In [22]:
wri = libsbml.SBMLWriter()

In [10]:
libsbml.writeSBML(yeast_document, 'test_merge.sbml')

1

In [ ]:
type()